In [ ]:
import pandas as pd
import regex as re

# From excel

In [ ]:
df = pd.read_excel("Newsweek.xlsx", index_col='id')

In [ ]:
df = df[df['source'] == 'Newsweek']
df = df[~df['text'].isna()]
df['magazine_nr'] = df['magazine_nr'].apply(lambda x: x.replace('_x000D_', '').strip())
df['magazine_nr'] = df['magazine_nr'].apply(lambda x: re.search("\d+/\d+", x).group(0) if re.search("\d+/\d+", x) is not None else None)

In [ ]:
df['title_main'] = df['title_main'].astype(str).fillna(' ')
df['title_sub'] = df['title_sub'].astype(str).fillna(' ')
df['title'] = df['title_main'] + ' ' +  df['title_sub']

In [ ]:
df = df[df['magazine_nr'].notna()]

# Date problem

In [ ]:
from bs4 import BeautifulSoup
import requests
mgz_list = []
date_list = []
for year in range(2015, 2023):
    print(year)
    url = f'https://www.newsweek.pl/lista-wydan/{year}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    divs = soup.find_all('div', {'class': 'editionsList__title'})

    texts = [re.search(r'\d.*\d', div.get_text()).group(0) for div in divs]
    
    dates  = [re.search(r'\/\s(.*)$', text).group(1) for text in texts]
    magazine_nrs = [re.search(r'^([\d\-]*)\s(.*)\.(\d+)$', text) for text in texts]
    magazine_nrs = [magazine_nr.group(1) + '/' + magazine_nr.group(3) for magazine_nr in magazine_nrs]
    mgz_list.extend(magazine_nrs)
    date_list.extend(dates)

In [ ]:
date_list.extend(['18.04.2017', '23.04.2018', '18.12.2018', '06.05.2019', '15.12.2019', '07.03.2020', '20.12.2020', '19.12.2021'])
mgz_list.extend(['18/2017', '18/2018', '52/2018', '17/2019', '51/2019', '11/2020', '52/2020', '51/2021'])
df_mapping = pd.DataFrame({'date': date_list, 'magazine_nr': mgz_list})

In [ ]:
df_merge = df.merge(df_mapping[['magazine_nr', 'date']], on='magazine_nr', how='left')
df_merge['date'] = df_merge['date_y'].fillna(df_merge['date_x'])
df_merge = df_merge.drop(['date_x', 'date_y'], axis=1)
df_error = df_merge[~df_merge['date'].str.contains(r'\.')]

In [ ]:
df_merge = df_merge.reset_index().rename(columns={'index':'id'})
df_merge.to_csv('newsweek.csv', index=False)

In [ ]:
df_merge['date_parsed'] = pd.to_datetime(df_merge['date'], format='%d.%m.%Y')
len(df_merge[df_merge['date_parsed'].isna()])